In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path

from pprint import pprint

import geopandas as gpd
import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import rasterio.features
import rasterio.plot
import rasterio.transform
import rasterio.warp
import shapely
import shapely.geometry as sg
from amr_terrain.backendInterface import amrBackend
import logging

logging.basicConfig(level=logging.INFO)
sys.executable, sys.path

# Read the YAML file

In [ ]:
yaml_file = Path('../yaml_files/13_austrian_alps.yaml').absolute()
assert yaml_file.is_file()
import yaml
cfg = yaml.safe_load(yaml_file.read_text(encoding="latin-1"))
cfg

In [ ]:
crs = rasterio.crs.CRS.from_user_input(cfg['caseCRS'])
crs.to_wkt().split('"')[1]

# Create the domain

In [ ]:

wm_coords = [['MM XXX', 500614.0, 5167720.0]]
wt_coords = [['SOB 02', 500854.0, 5167284.0], ['SOB 01', 500648.0, 5167642.0]]
wm_wt_distance = 4_000
domain_distance = 10_000

objects = pd.DataFrame(wm_coords + wt_coords, columns=["Name", "X", "Y"])
objects["geometry"] = gpd.points_from_xy(objects.X, objects.Y)
objects = gpd.GeoDataFrame(objects, crs="EPSG:32633").to_crs(crs)
objects

In [ ]:
domain_center = objects.to_crs("EPSG:4326").union_all().centroid
centerLon, centerLat = domain_center.x, domain_center.y
centerLat, centerLon

In [ ]:
objects_ref_buffer = 4_000
domain_buffer_buffer = 6_000
max_ref_level = 3
horizontal_spacing = 25
vertical_spacing = 5
vertical_ar = horizontal_spacing / vertical_spacing
coarse_horizontal_spacing = horizontal_spacing * 2**max_ref_level

print(f"Fine grid spacing: {horizontal_spacing} m at level {max_ref_level}.")
print(f"Level 0 grid spacing: {coarse_horizontal_spacing} m")


ref_zone = gpd.GeoSeries(objects.buffer(objects_ref_buffer).union_all())
domain_zone = ref_zone.buffer(domain_buffer_buffer)

In [ ]:
tiff_filename = Path(cfg["useTiff"])
if not tiff_filename.is_file():
    tiff_filename = yaml_file.parent.joinpath("elevation_dtm_austria.tif")
print(tiff_filename)
with rasterio.open(tiff_filename) as tif_in:
    tif_bounds = gpd.GeoSeries(sg.box(*tif_in.bounds), crs=tif_in.crs).to_crs(crs)
    available_terrain_area = sg.box(*tif_bounds.total_bounds)
tif_bounds


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=96)
tif_bounds.plot(ax=ax, facecolor=(0, 0, 0, 0), linewidth=2, linestyle="--", edgecolor="k", label="GeoTiff bound", zorder=30)
gpd.GeoSeries(sg.box(*domain_zone.total_bounds)).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="red", label="Domain", hatch="////", zorder=40)
gpd.GeoSeries(sg.box(*ref_zone.total_bounds)).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="darkgreen", label="Refinement zone", linewidth=2, zorder=50)
objects.plot(
    ax=ax,
    label="Objects",
    color="darkgreen",
    lw=0,
    marker="o",
    zorder=500,
)
tif_bounds.plot(ax=ax, facecolor=(0, 0, 0, 0), linewidth=2, linestyle="--", edgecolor="k", label="GeoTiff bound", zorder=30)
ax.tick_params(labelsize="x-small")
fig.tight_layout()
IPython.display.display(fig)
plt.close(fig)

# Evaluate the fringes

In [ ]:
import math
from rasterio.coords import BoundingBox
def nice_bounds(bounds: list[float], resolution) -> BoundingBox:
    west, south, east, north = bounds
    west = math.floor(west / resolution) * resolution
    south = math.floor(south / resolution) * resolution
    east = math.ceil(east / resolution) * resolution
    north = math.ceil(north / resolution) * resolution
    return BoundingBox(west, south, east, north)

domain_bounds = nice_bounds(
    domain_zone.total_bounds,
    coarse_horizontal_spacing,
)
domain_area = sg.box(*domain_bounds)
assert available_terrain_area.contains(domain_area), (
    "The domain area is not contained by the elevation raster",
    domain_area,
    available_terrain_area,
)
domain_area.bounds


In [ ]:
import rasterio.vrt
from rasterio.enums import Resampling
with rasterio.open(tiff_filename) as src:
    with rasterio.vrt.WarpedVRT(src, crs=crs, resampling=Resampling.bilinear) as vrt:
        elevation_data = vrt.read(1, masked=True).astype("float")
        elevation_transform = vrt.transform
        
elevation_data.shape, elevation_transform

In [ ]:
def retrieve_edges_information(bounds, z_data, transform):
    import rasterio.features
    from shapely import LineString
    
    x_min, y_min, x_max, y_max = bounds
    res = max(
        transform.a,
        -transform.e,
    )

    def get_min_max_for_shape(shp):
        im = (
            rasterio.features.rasterize(
                [(shp.buffer(res), 1)],
                out_shape=z_data.shape,
                fill=0,
                transform=transform,
                all_touched=True,
                merge_alg=rasterio.features.MergeAlg.replace,
                masked=True,
            )
            == 1
        )
        return [float(np.amin(z_data[im]).round(1)), float(np.amax(z_data[im]).round(1))]

    
    domain_edges_z_min_max = []

    # west
    domain_edges_z_min_max.append(
        get_min_max_for_shape(LineString([(x_min, y_min), (x_min, y_max)]))
    )
    # east
    domain_edges_z_min_max.append(
        get_min_max_for_shape(LineString([(x_max, y_min), (x_max, y_max)]))
    )
    # south
    domain_edges_z_min_max.append(
        get_min_max_for_shape(LineString([(x_min, y_min), (x_max, y_min)]))
    )
    # north
    domain_edges_z_min_max.append(
        get_min_max_for_shape(LineString([(x_min, y_max), (x_max, y_max)]))
    )

    return domain_edges_z_min_max


domain_edges_z_min_max = retrieve_edges_information(domain_area.bounds, elevation_data, elevation_transform)
domain_edges_z_min_max



# Evaluate the fringes

In [ ]:
im = (
    rasterio.features.rasterize(
        [(domain_area, 1)],
        out_shape=elevation_data.shape,
        fill=0,
        transform=elevation_transform,
        all_touched=True,
        merge_alg=rasterio.features.MergeAlg.replace,
        masked=True,
    )
    == 1
)
terrain_z_min = float(np.amin(elevation_data[im]))
terrain_z_max = float(np.amax(elevation_data[im]))

print(f"terrain_z_min: {terrain_z_min:.1f} m")
print(f"terrain_z_max: {terrain_z_max:.1f} m")

terrain_delta_z = terrain_z_max - terrain_z_min
print(f"terrain_delta_z: {terrain_delta_z:.1f} m")
rdl_height = max(terrain_delta_z, 2048)
abl_height = max(terrain_delta_z, 2048)
domain_max_z = terrain_delta_z + abl_height + rdl_height


domain_edges_z_max = [_[1] for _ in domain_edges_z_min_max]

def find_transition_length(vertical_length, grid_spacing: float = 100, max_transition_slope=20.0/100) -> None:
    tr_length = grid_spacing
    while (vertical_length / tr_length) >= max_transition_slope:
        tr_length += grid_spacing
    tr_length = (
        np.ceil(tr_length / grid_spacing) * grid_spacing
    )
    return float(tr_length)

west_slope = find_transition_length(
    domain_edges_z_max[0] - terrain_z_min,
    grid_spacing=coarse_horizontal_spacing,
    max_transition_slope=20.0/100,
)
print(f"westSlope: {west_slope:.1f}")
south_slope = find_transition_length(
    domain_edges_z_max[1] - terrain_z_min,
    grid_spacing=coarse_horizontal_spacing,
    max_transition_slope=20.0/100,
)
print(f"southSlope: {south_slope:.1f}")
east_slope = find_transition_length(
    domain_edges_z_max[2] - terrain_z_min,
    grid_spacing=coarse_horizontal_spacing,
    max_transition_slope=20.0/100,
)
print(f"eastSlope: {east_slope:.1f}")
north_slope = find_transition_length(
    domain_edges_z_max[3] - terrain_z_min,
    grid_spacing=coarse_horizontal_spacing,
    max_transition_slope=20.0/100,
)
print(f"northSlope: {north_slope:.1f}")

x_min, y_min, x_max, y_max = domain_area.bounds
x_min -= west_slope
x_max += east_slope
y_min -= south_slope
y_max += north_slope
slope_area = sg.box(x_min, y_min, x_max, y_max)
slope_area.bounds

In [ ]:
def get_grid_padding(bounds, dxy, dz, domain_max_z, blocking_factor=8):
    x_min, y_min, x_max, y_max = bounds
    lx = x_max - x_min
    ly = y_max - y_min
    nx_i = nx = int(lx / dxy)
    ny_i = ny = int(ly / dxy)
    while nx % blocking_factor != 0:
        nx = nx + 1
    while ny % blocking_factor != 0:
        ny = ny + 1
    nz_i = nz = int(domain_max_z / dz)
    while nz % blocking_factor != 0:
        nz = nz + 1

    added_lx = (nx * dxy) - lx
    added_ly = (ny * dxy) - ly
    added_lz = (nz * dz) - domain_max_z
    return added_lx, added_ly, added_lz


flat_length = 2_000

x_min, y_min, x_max, y_max = nice_bounds(
    slope_area.buffer(flat_length).bounds,
    coarse_horizontal_spacing,
)
added_lx, added_ly, _ = get_grid_padding([x_min, y_min, x_max, y_max], coarse_horizontal_spacing, coarse_horizontal_spacing / vertical_ar, domain_max_z, blocking_factor=8)
print((added_lx, added_ly))
x_min -= added_lx / 2
x_max += added_lx / 2
y_min -= added_ly / 2
y_max += added_ly / 2
flat_area =  sg.box(x_min, y_min, x_max, y_max)

flat_west = slope_area.bounds[0] - x_min
flat_south = slope_area.bounds[1] - y_min
flat_east = x_max - slope_area.bounds[2] 
flat_north = y_max - slope_area.bounds[3] 

print(f"westFlat: {flat_west:.1f}")
print(f"eastFlat: {flat_east:.1f}")
print(f"southFlat: {flat_south:.1f}")
print(f"northFlat: {flat_north:.1f}")

In [ ]:
x_min, y_min, x_max, y_max = flat_area.bounds
lx = x_max - x_min
ly = y_max - y_min
nx = int(lx/coarse_horizontal_spacing)
ny = int(ly/coarse_horizontal_spacing)
assert nx * coarse_horizontal_spacing == lx
assert ny * coarse_horizontal_spacing == ly
print(f"LX: {lx:.1f}")
print(f"LY: {lx:.1f}")
print(f"NX: {nx:.1f}")
print(f"NY: {ny:.1f}")
print("\n"*4)



center_pt = domain_zone.union_all().centroid
west = center_pt.x - x_min
east = x_max - center_pt.x
south = center_pt.y - y_min
north = y_max - center_pt.y


print("=====  YAML INPUT  =====") 
print(f"useTiff: {str(tiff_filename)}")
print("")

print("# Domain area")
print(f"centerLat: {centerLat:.8f}")
print(f"centerLon: {centerLon:.8f}")
print("")
print(f"cellSize: {coarse_horizontal_spacing:.0f}")
print(f"verticalAR: {vertical_ar:.0f}")
print("")

print(f"west: {west:.3f}")
print(f"south: {south:.3f}")
print(f"east: {east:.3f}")
print(f"north: {north:.3f}")

print("# Slope area")
print(f"westSlope: {west_slope:.1f}")
print(f"southSlope: {south_slope:.1f}")
print(f"eastSlope: {east_slope:.1f}")
print(f"northSlope: {north_slope:.1f}")
print("# Flat area")
print(f"westFlat: {flat_west:.1f}")
print(f"eastFlat: {flat_east:.1f}")
print(f"southFlat: {flat_south:.1f}")
print(f"northFlat: {flat_north:.1f}")



In [ ]:
import rasterio.plot
fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=96)
tif_bounds.plot(ax=ax, facecolor=(0, 0, 0, 0), linewidth=2, linestyle="--", edgecolor="k", label="GeoTiff bound", zorder=30)
im = ax.imshow(elevation_data, extent=rasterio.plot.plotting_extent(elevation_data, elevation_transform), cmap="terrain", zorder=5, alpha=0.4)
gpd.GeoSeries(sg.box(*domain_zone.total_bounds)).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="red", label="Domain", hatch="////", zorder=40)
gpd.GeoSeries(sg.box(*ref_zone.total_bounds)).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="darkgreen", label="Refinement zone", linewidth=2, zorder=50)
gpd.GeoSeries(domain_area).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="darkgreen", label="Refinement zone", linewidth=2, zorder=50)
gpd.GeoSeries(slope_area).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="darkgreen", label="Slope area", linewidth=2, zorder=60)
gpd.GeoSeries(flat_area).plot(ax=ax, facecolor=(0, 0, 0, 0), edgecolor="darkgreen", label="Flat area", linewidth=2, zorder=70)
objects.plot(
    ax=ax,
    label="Objects",
    color="darkgreen",
    lw=0,
    marker="o",
    zorder=500,
)

tif_bounds.plot(ax=ax, facecolor=(0, 0, 0, 0), linewidth=2, linestyle="--", edgecolor="k", label="GeoTiff bound", zorder=30)
ax.tick_params(labelsize="x-small")
fig.tight_layout()
IPython.display.display(fig)
plt.close(fig)

# Create the backed

In [ ]:
import contextlib
ROOT_PATH = yaml_file.resolve().absolute().parent.parent
print(ROOT_PATH)

with contextlib.chdir(ROOT_PATH):
    backend = amrBackend(yaml_file)
    print(Path(backend.caseParent, backend.caseName).resolve().absolute())
    
print(backend.caseCRS.to_wkt().split('"')[1])

case_path = Path(backend.caseParent).joinpath(backend.caseName)
list(case_path.glob("*"))

In [ ]:
case_center = rasterio.warp.transform(
    "EPSG:4326",
    backend.caseCRS, 
    [backend.caseCenterLon], 
    [backend.caseCenterLat], 
)
case_center = sg.Point(case_center[0], case_center[1])
case_center.xy

In [ ]:
from functools import partial
utm_to_cfd = partial(
    shapely.transform,
    transformation=lambda pt: pt  - np.asarray(case_center.xy).flatten(), include_z=False
)
cfd_to_utm = partial(
    shapely.transform,
    transformation=lambda pt: pt  + np.asarray(case_center.xy).flatten(), include_z=False
)

domain_cfd = sg.box(
    -backend.caseEast,
    -backend.caseSouth,
    backend.caseWest,
    backend.caseNorth,
)
domain_proj = cfd_to_utm(domain_cfd)
domain_proj.bounds

In [ ]:
from importlib.resources import files

if 0:
    turbines_path = files("amr_terrain").joinpath("turbine.csv")
    assert turbines_path.is_file()


    turbines_df = pd.read_csv(turbines_path, sep=",", header=0, encoding="latin-1")


    turbines_df["geometry"] = gpd.points_from_xy(turbines_df.xlong, turbines_df.ylat)
    turbines_df = gpd.GeoDataFrame(
        turbines_df, crs="EPSG:4326"
    ).to_crs(backend.caseCRS)

    turbines_df = turbines_df[turbines_df.intersects(domain_proj.buffer(-5_000))]
    turbines_df.p_name.value_counts()
    ax = turbines_df.plot(column="p_name")
    gpd.GeoSeries([domain_proj.exterior]).plot(ax=ax, color="k", linestyle="--")

# Creating the domain

In [ ]:
with contextlib.chdir(ROOT_PATH):
    backend.createDomain()
    if backend.write_stl:
        plt.close("all")
backend.srtm_output   

# Get the domain dimensions

In [ ]:
backend.createAMRGeometry(None)
nx_b, ny_b, nz_b = backend.createAMRGrid(None)
assert nx_b == nx
assert ny_b == ny
xs, ys, elev, z_interp, transform = backend.createElevationInterpolator(as_cfd=True)
transform


In [ ]:
import rasterio.plot
extent = rasterio.plot.plotting_extent(elev, transform) 

if 1:
    fig, ax = plt.subplots(figsize=(4, 4), dpi=48)
    # cm = ax.pcolormesh(xs, ys, elev, cmap="terrain")  # ,vmin=vmin,vmax=vmax)
    cm = ax.imshow(elev, cmap="terrain", extent=extent, origin="lower")
    cb = fig.colorbar(cm, ax=ax)
    cb.set_label("elevation [m]", fontsize="x-large")
    ax.tick_params(labelsize="large")
    ax.set_xlabel("easting [m]")
    ax.set_ylabel("northing [m]")
    ax.set_title("shifted terrain height")
    ax.axis("scaled")

if 1:
    fig, ax = plt.subplots(figsize=(4, 4), dpi=48)
    cm = ax.pcolormesh(xs, ys, elev, cmap="terrain")  # ,vmin=vmin,vmax=vmax)
    cb = fig.colorbar(cm, ax=ax)
    cb.set_label("elevation [m]", fontsize="x-large")
    ax.tick_params(labelsize="large")
    ax.set_xlabel("easting [m]")
    ax.set_ylabel("northing [m]")
    ax.set_title("shifted terrain height")
    ax.axis("scaled")


if 1:
    xsurf, ysurf = np.meshgrid(xs, ys, indexing="ij")
    fig, ax = plt.subplots(figsize=(4, 4), dpi=48)
    cm = ax.pcolormesh(xsurf, ysurf, z_interp(xs, ys, grid=True), cmap="terrain")
    cb = fig.colorbar(cm, ax=ax)
    cb.set_label("elevation [m]", fontsize="x-large")
    ax.tick_params(labelsize="large")
    ax.set_xlabel("easting [m]")
    ax.set_ylabel("northing [m]")
    ax.axis("scaled")
    del xsurf, ysurf